In [1]:
import requests
from catsim.cat import generate_item_bank
from catsim.simulation import *
from catsim.initialization import *
from catsim.selection import *
from catsim.estimation import *
from catsim.stopping import *
import catsim.plot as catplot
from catsim.irt import icc
import matplotlib.pyplot as plt
from NODO import NODO
from RED_BAYESIANA import RED_BAYESIANA
from MPCN import MPCN
import pickle
import os.path
from os import path
import pyAgrum as gum

URL = "http://159.223.190.216/arbol/caminoslibreria"
respuestaDelServidor = requests.get(url = URL).json()
caminos = respuestaDelServidor['caminos']
CONTRUCTOR_BN = RED_BAYESIANA()

id_alumno = 5

saltos = 3
tema_objetivo = 185
ponderacion_objetiva = 90

nombre_rbma = "rbma_xml/rbma_alumno_"+str(id_alumno)+".bifxml"
nombre_rbma_pickle = "rbma_pickle/rbma_alumno_"+str(id_alumno)+".pkl" #pickle
nombre_rmb = "rbm/rbm_1.bifxml"
CONTRUCTOR_BN.configurarRBM(nombre_rmb) # Asignamos el nombre de la Red Bayesiana Maestra a utilizar

RBMA = {}
RBMA_XML = ""
if not path.exists(nombre_rbma_pickle):
    RBMA_XML = CONTRUCTOR_BN.obtenerRedCompleta() # se copia la RBM

    for nodo in RBMA_XML.nodes():
        nodo_rbm = NODO(RBMA_XML.variable(nodo).name(), 1)
        for tema in respuestaDelServidor['temas']:
            if RBMA_XML.variable(nodo).name() == str(tema['id']):
                nodo_rbm.dificultad = tema['dificultad']
        RBMA[RBMA_XML.variable(nodo).name()] = nodo_rbm
    # Almacenar la RBMA de manera persistente
    with open(nombre_rbma_pickle, 'wb') as outp:
        pickle.dump( RBMA, outp, pickle.HIGHEST_PROTOCOL)
    gum.saveBN(RBMA_XML, nombre_rbma)
else:
    with open(nombre_rbma_pickle, 'rb') as inp:
        RBMA = pickle.load(inp)
    RBMA_XML = gum.loadBN(nombre_rbma)

CONTRUCTOR_BN.configurarParametros([tema_objetivo,ponderacion_objetiva,saltos])
SRB = CONTRUCTOR_BN.obtenerSubRed() # 508
CAMINOS_SRB = CONTRUCTOR_BN.obtenerCaminosSubRed(tema_objetivo, saltos, caminos)

INSTANCIA_MPCN = MPCN(RBMA, CAMINOS_SRB, tema_objetivo, ponderacion_objetiva)
INSTANCIA_MPCN.ponderarNodos() #
INSTANCIA_MPCN.hacerInferencias(RBMA_XML)
INSTANCIA_MPCN.promediarInferencias(SRB)

In [2]:
for nodo in RBMA:
    print(RBMA[nodo].id)
    print(RBMA[nodo].matriz_inferencia_final)
    print(RBMA[nodo].matriz_inferencia)
    print("==================")

98
[0, 0, 0]
[26.361311272528702, 37.790618466807075, 35.84807026066423]
178
[0, 0, 0]
[18.77084719799114, 38.744111626660626, 42.485041175348236]
179
[0, 0, 0]
[23.233243422269283, 31.848107114925966, 44.918649462804744]
180
[0, 0, 0]
[32.89474260554943, 32.18680511141987, 34.91845228303071]
32
[0, 0, 0]
[31.06672628298821, 28.2195651672749, 40.71370854973688]
96
[0, 0, 0]
[38.119077198517374, 33.94372145220378, 27.93720134927885]
97
[0, 0, 0]
[12.050168189274169, 19.161888416779085, 68.78794339394675]
169
[0, 0, 0]
[41.9346394870321, 30.33412597096682, 27.731234542001086]
177
[0, 0, 0]
[25.162089050549223, 33.81259941196632, 41.02531153748446]
31
[0, 0, 0]
[39.96576032504458, 27.997084540577553, 32.03715513437786]
95
[0, 0, 0]
[24.28032887840869, 28.360211480129855, 47.35945964146145]
101
[0, 0, 0]
[26.176649735711255, 35.20556533484252, 38.61778492944623]
168
[0, 0, 0]
[32.31044921656902, 30.037749966085634, 37.651800817345354]
175
[0, 0, 0]
[27.130012831130575, 30.740586322255204, 

In [3]:

RBMA = INSTANCIA_MPCN.obtenerNodos() # Se obtiene la RBMA actualizada con las inferenicas promediadas de la SRB
items = INSTANCIA_MPCN.obtenerItems() # Calcula las caracteristicas de Seudoadivinacion, Discriminzacion y Dificultad

In [4]:
items

array([[ 1.        , -1.97142857,  0.3584807 ,  1.        ,  0.        ],
       [ 1.        , -0.25714286,  0.42485041,  1.        ,  0.        ],
       [ 1.        , -0.25714286,  0.44918649,  1.        ,  0.        ],
       [ 1.        , -0.25714286,  0.34918452,  1.        ,  0.        ],
       [ 1.        , -2.05714286,  0.40713709,  1.        ,  0.        ],
       [ 1.        , -2.14285714,  0.27937201,  1.        ,  0.        ],
       [ 1.        , -2.05714286,  0.68787943,  1.        ,  0.        ],
       [ 1.        , -1.11428571,  0.27731235,  1.        ,  0.        ],
       [ 1.        , -0.34285714,  0.41025312,  1.        ,  0.        ],
       [ 1.        , -2.14285714,  0.32037155,  1.        ,  0.        ],
       [ 1.        , -2.22857143,  0.4735946 ,  1.        ,  0.        ],
       [ 1.        , -2.31428571,  0.38617785,  1.        ,  0.        ],
       [ 1.        , -1.2       ,  0.37651801,  1.        ,  0.        ],
       [ 1.        , -0.42857143,  0.4

In [6]:
temas_evaluados = []
temas_evaluados_ids = []
respuestas = []
for nodo in RBMA:
    if RBMA[nodo].clasificacion == 3:
        temas_evaluados.append(0)
        temas_evaluados_ids.append(0)
        respuestas.append(0)
i=0
for nodo in RBMA:
    if RBMA[nodo].clasificacion == 3:
        temas_evaluados_ids[RBMA[nodo].orden_evaluacion] =  RBMA[nodo].id
        temas_evaluados[RBMA[nodo].orden_evaluacion] = i
        if RBMA[nodo].ponderacion > 70:
            respuestas[RBMA[nodo].orden_evaluacion] = True
        else:
            respuestas[RBMA[nodo].orden_evaluacion] = False
    i=i+1

IndexError: list assignment index out of range

In [10]:
print(temas_evaluados_ids)
print(temas_evaluados)
print(respuestas)

[0]
[0]
[0]


In [ ]:
temas_evaluados = ['76', '186', '185']

In [9]:
initializer = RandomInitializer()
selector = MaxInfoSelector()
estimator = NumericalSearchEstimator()
stopper = MaxItemStopper(20)
#est_theta = initializer.initialize()
est_theta = 0.0
new_theta = estimator.estimate(items=items, administered_items=temas_evaluados, response_vector=respuestas, est_theta=est_theta)
_stop = stopper.stop(administered_items=items[temas_evaluados], theta=est_theta)
sig_tema = selector.select(items=items, administered_items=temas_evaluados, est_theta=est_theta)

print("NODOS EVALUADOS ==================================")
for evaluado in temas_evaluados:
    print("ID NODO: " + str( RBMA[ list(RBMA)[evaluado] ].id ) )
    print("DIFICULTAD: " + str( RBMA[ list(RBMA)[evaluado] ].dificultad ) )
    print("RESPUESTA: " + str(respuestas[ temas_evaluados.index(evaluado) ]))
    print("_____________________")
print("NODO SIGUIENTE ==================================")
print("ID NODO: " + str(  RBMA[ list(RBMA)[sig_tema] ].id )  )
print("DIFICULTAD: " + str(  RBMA[ list(RBMA)[sig_tema] ].dificultad )  )

with open(nombre_rbma_pickle, 'wb') as outp:
    pickle.dump( RBMA, outp, pickle.HIGHEST_PROTOCOL)

gum.saveBN(RBMA_XML, nombre_rbma)

NODOS EVALUADOS ==================================
ID NODO: 98
DIFICULTAD: -1.97142857
RESPUESTA: 0
_____________________
NODO SIGUIENTE ==================================
ID NODO: 186
DIFICULTAD: -0.17142857
